In [2]:
# sets up the pyserini sparse index
!python3 -m pyserini.index.lucene --collection JsonCollection --input data/websites --index data/pyserini_index --generator DefaultLuceneDocumentGenerator --threads 1

2022-04-10 17:59:31,441 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-10 17:59:31,443 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-10 17:59:31,443 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-10 17:59:31,444 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: data/websites
2022-04-10 17:59:31,444 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-10 17:59:31,444 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-10 17:59:31,444 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 1
2022-04-10 17:59:31,445 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-10 17:59:31,445 INFO  [main] index.IndexCollection (IndexCol

In [4]:
!python3 -m pyserini.search.lucene --index data/pyserini_index --topics data/bow_queries_dev.tsv --output out/bm25_runs/run.dev.txt --bm25 --k1 3 --b 0.9

# note MAP = MRR when there is exactly one relevant result
# https://stats.stackexchange.com/questions/127041/mean-average-precision-vs-mean-reciprocal-rank
!python3 -m pyserini.eval.trec_eval -m map -m P.1 data/relevance_scores.txt out/bm25_runs/run.dev.txt

Setting BM25 parameters: k1=3.0, b=0.9
Running data/bow_queries_dev.tsv topics, saving to out/bm25_runs/run.dev.txt...
100%|█████████████████████████████████████████| 667/667 [00:28<00:00, 23.41it/s]
/home/kjros2/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/home/kjros2/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-m', 'map', '-m', 'P.1', 'data/relevance_scores.txt', 'out/bm25_runs/run.dev.txt']
Results:
map                   	all	0.3719
P_1                   	all	0.2744

